In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_core.prompts import  ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS 
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader 

In [3]:
loader = PyPDFLoader("data/Muet_Prospectus-23.pdf")
loaded_document = loader.load()

In [4]:
loaded_document

[Document(metadata={'source': 'data/Muet_Prospectus-23.pdf', 'page': 0}, page_content=' \n  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n   \n \n \n \nPROSPECTUS  \n2023 -24 '),
 Document(metadata={'source': 'data/Muet_Prospectus-23.pdf', 'page': 1}, page_content=' \n \n i \n  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nACKNOWLEDGMENT  \n \nWe are thankful  to stakeholders for their involvement  in preparation of this Prospectus.  \n \nDisclaimer  \n \nThe information in this prospectus is correct at the time of publishing. The University reserves the \nright to add or remove courses and to make changes in Syllabi , Courses Options and Modules, Fees \netc. at any stage . Although  every effort is made to ensure accuracy at the time of publication, \nUniversity reserves the right to make any corrections in the contents and provisions without notice. \nFor further information please contact admissions@admin.muet.edu.pk  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n 

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=5)
chunks = text_splitter.split_documents(loaded_document)

In [6]:
chunks

[Document(metadata={'source': 'data/Muet_Prospectus-23.pdf', 'page': 0}, page_content='PROSPECTUS  \n2023 -24'),
 Document(metadata={'source': 'data/Muet_Prospectus-23.pdf', 'page': 1}, page_content='i \n  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nACKNOWLEDGMENT  \n \nWe are thankful  to stakeholders for their involvement  in preparation of this Prospectus.  \n \nDisclaimer  \n \nThe information in this prospectus is correct at the time of publishing. The University reserves the \nright to add or remove courses and to make changes in Syllabi , Courses Options and Modules, Fees \netc. at any stage . Although  every effort is made to ensure accuracy at the time of publication, \nUniversity reserves the right to make any corrections in the contents and provisions without notice. \nFor further information please contact admissions@admin.muet.edu.pk'),
 Document(metadata={'source': 'data/Muet_Prospectus-23.pdf', 'page': 2}, page_content="ii \n \n MEHRAN UNIVERSITY OF  ENGINEERING \n&TECHN

In [7]:
len(chunks)

208

In [8]:
embeddings = OpenAIEmbeddings()

vector_db = FAISS.from_documents(chunks, embeddings)

In [9]:
vector_db

In [10]:
retriever = vector_db.as_retriever()

In [11]:
response = retriever.invoke("who is the chairman of Software Department?")

In [12]:
response

[Document(metadata={'source': 'data/Muet_Prospectus-23.pdf', 'page': 41}, page_content="Page No. 35 \n 3.5 Department of Software Engineering  \n  \n3.5.1  The Department  \nThe Department of Software Engineering is home to research and academic units that address issues and \nrecent advances in software engineering. The department provides research areas and cutting -edge facilities \nin Bachelor of Engineering in Software Engineering and Bachelor of Science in Artificial Intelligence. The \ngoal has been, and continues to be, to provide a high -degree  program in Software Engineering and Artificial \nIntelligence that prepares students for lifelong learning as they take on professional careers in the software \nindustry. The degree programs enable students to gain a thorough understanding of the role of sof tware \ndevelopment and artificial intelligence in enterprise organizations and to transform conventional systems \ninto digitization that improves business and organizational pro

In [13]:
retriever = vector_db.as_retriever(search_kwargs={"k":1})

In [14]:
response = retriever.invoke("who is the chairman of Software Department?")

In [15]:
response[0].page_content

"Page No. 35 \n 3.5 Department of Software Engineering  \n  \n3.5.1  The Department  \nThe Department of Software Engineering is home to research and academic units that address issues and \nrecent advances in software engineering. The department provides research areas and cutting -edge facilities \nin Bachelor of Engineering in Software Engineering and Bachelor of Science in Artificial Intelligence. The \ngoal has been, and continues to be, to provide a high -degree  program in Software Engineering and Artificial \nIntelligence that prepares students for lifelong learning as they take on professional careers in the software \nindustry. The degree programs enable students to gain a thorough understanding of the role of sof tware \ndevelopment and artificial intelligence in enterprise organizations and to transform conventional systems \ninto digitization that improves business and organizational processes.  \n \nThe department offers a range of courses in both degree programs in acco 

In [16]:
len(response)

1

In [17]:
# using LCEL 
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [18]:
# context = "This is the context information that will be used to answer the questions."

template = """Answer appropriately for the following questions based on the following context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [19]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()  
)

In [20]:
response = chain.invoke("Who is the chairman of the Software Department?")

In [21]:
response

'The chairman of the Software Department is Dr. Naeem Ahmed Mahoto.'

In [22]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=1)
search.invoke("Who is the current Chairman of the Software Department of Mehran University of Engineering and Technology, Jamshoro?")

[{'url': 'https://sw.muet.edu.pk/faculty.php',
  'content': 'Department of Software Engineering B.E Software Engineering B.S Artificial Intellingence B.E Software Engineering Curriculum Contents B.E Software Engineering Course Scheme B.E Software Engineering Timetable Faculty Chairman/Professor Associate Professor Dr. Mohsin Memon Associate Professor Assistant Professor Assistant Professor Assistant Professor Ms. Amirita Dewani Assistant Professor Dr. Anoud Shaikh Assistant Professor Assistant Professor Assistant Professor Assistant Professor Assistant Professor Ms. Mariam Memon Mr. Mansoor Ms. Mehwish Shaikh Ms. Fahama Barakzai Ms. Fatima Ms. Sana Faiz Faculty on Leave Associate Professor Assistant Professor Ms. Hira Noman Assistant Professor Assistant Professor Ms. Rabia Iftikhar Contact*   Department of Software Engineering, Mehran University of Engineering & Technology, Jamshoro, Sindh, Pakistan - 76062 ©2018 Department of Software Engineering, MUET | ICT Building | Jamshoro, 76062

In [23]:
tools = [search]

In [24]:
tools 

[TavilySearchResults(max_results=1)]

In [25]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [26]:
from langgraph.prebuilt import create_react_agent 

agent_executor = create_react_agent(llm, tools)

In [27]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="who is the current Vice Chancellor of MUET?")]})

response["messages"]

[HumanMessage(content='who is the current Vice Chancellor of MUET?', id='0cc85a07-eb88-4521-8a76-c06a78190c63'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_H5DvTGkCTf3IWBCVBzlPeWrr', 'function': {'arguments': '{"query":"current Vice Chancellor of Mehran University of Engineering and Technology"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 91, 'total_tokens': 120, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-98d6e181-b6ce-4a1b-a447-7e44881cc5da-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current Vice Chancellor of Mehran University of Engineering and Technol

In [28]:
for chunk in agent_executor.stream(
    {'messages': [HumanMessage(content="What is the rank of MUET globally and in Pakistan? ")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3xlsBVj64ivGSn8lQWZncASp', 'function': {'arguments': '{"query": "MUET global ranking"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_VcP7EAoR3PseomMDROS9Yi1b', 'function': {'arguments': '{"query": "MUET Pakistan ranking"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 94, 'total_tokens': 152, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d0e23b40-745f-45f6-a3e8-0ff44f46cea1-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'MUET global ranking'}, 'id': 'call_3xlsBVj64ivGSn8l

## **Adding Memory**

In [29]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [30]:
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

config = {"configurable" : {"thread_id": "001"}}

In [31]:
for chunk in agent_executor.stream(
    {'messages': [HumanMessage(content="what do you know about the faculty of Software Department of MUET?")]}, config
    
):
    print(chunk)
    print('----')

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_MLKnblgcNUQsjCAAgmShvxwT', 'function': {'arguments': '{"query":"Faculty of Software Department of MUET"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 95, 'total_tokens': 120, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1ab0ae6b-7816-4fb4-97e8-ea6e9d833d16-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Faculty of Software Department of MUET'}, 'id': 'call_MLKnblgcNUQsjCAAgmShvxwT', 'type': 'tool_call'}], usage_metadata={'input_tokens': 95, 'output_tokens': 25, 'total_tokens': 120})]}}
----
{'tool

In [32]:
config = {"configurable": {"thread_id": "001"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="which what we were talking about?")]}, config
):
    print(chunk)
    print('-----')

{'agent': {'messages': [AIMessage(content='We were discussing the faculty members of the Software Department at Mehran University of Engineering & Technology (MUET).', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 527, 'total_tokens': 551, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1e0fa4fd-bcde-42f1-ac6b-5869d31269c1-0', usage_metadata={'input_tokens': 527, 'output_tokens': 24, 'total_tokens': 551})]}}
-----
